# Week 2: Introduction to Principal Component Analysis (PCA)

## Tutorial

In this week, we will study **dimensionality reduction**, and more specifically about **principal component analysis (PCA)**.

In this module you will learn:

1. sketch the direction of the first principal component for some given data in a 2-D space
2. explain why it is crucial to standardize the data before applying PCA
3. apply PCA in Python using sci-kit learn
4. adjust PCA parameters for your specific use case

### Dimensionality Reduction

*What is dimensionality reduction?*

Dimensionality reduction is a technique to reduce the number of features (characteristics, or input variables, etc.) of a dataset, while preserving the most important information.  

*Why would we ever need to reduce the number of features in a dataset?*

Let's look at an example. Suppose that we are analyzing a data set of breast cancer histopathology images. We have provided two images below. The left image contains benign breast cancer issues, whereas the right image contains malignant breast cancer tissues. Our goal is to determine whether an image contains benign or malignant breast cancer issues. How would you perform this prediction?



![benign-malignant.jpg](download.jpeg)

*How do we predict whether an image contains benign or malignant cancer issues?*

You might notice that benign and breast cancer tissues have distinct characteristics, such as different ductal structures, nuclear morphologies, cell morphologies, and so on.

In a computer, each image is represented as a 2-D grid of pixels. Each pixel provides a tiny piece of information through its color. You can think of the pixels in an image encoding some information about the breast cancer issue.
We often consider each pixel of an image as a **feature**. The value of each feature is the color of the pixel converted to a number. (You may already be familiar with the term **features** under another name, "variables".)

Although each image has many pixels, only some pixels are important for determining whether the breast cancer issue is benign or malignent. We want to figure out and isolate the important pixels. Our ultimate goal is to preserve all the relevant features (pixels) that are required for classification and remove all the rest, ultimately reducing the dimensions of the data.

### Data Exploration

Analyzing actual breast cancer histopathology images would require a lot of compute power since these images are **extremely** high dimensional (i.e. have a lot of pixels). Instead, we're going to look at the breast cancer dataset from sklearn. This dataset contains features extracted from images of breast cancer biopsies. This data is still high-dimensional, but it is feasible for us to perform the analysis and interpret the results.

To begin let's import the necessary libraries.


In [1]:
import numpy as np  # for numerical operations
import pandas as pd # for data manipulation and analysis

import matplotlib.pyplot as plt   #  for plotting
import matplotlib as mpl    # for configurating the plotting
mpl.rcParams["axes.spines.right"] = False
mpl.rcParams["axes.spines.top"] = False

# sklearn for machine learning
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# we'll use the breast cancer dataset as an example of high-dimensional biological data
data = load_breast_cancer()

Features of an actual breast cancer image are pixels of the image. The sklearn dataset loaded provides less features for each breast cancer sample. Let's look at a description of the sklearn breast cancer dataset.

In [ ]:
print(data["DESCR"])

---
##### **Q1. How many features does the dataset have? Give 4 examples of the features.**

<span style="background-color: #FFD700">**Write Answer Here**</span>

---


Even though 30 features is much less than the number of pixels in an image, they are difficult to visualize 30 dimensions. Instead, we can use PCA to bring the data down into two dimensions, which we can visualize on a plot.

### Explaining Principal Components

Principal Component Analysis (PCA) aims to preserve the features that "explain the most variance". We would therefore expect the principal components that the data is reduced to, to correspond to the biggest "spread" or variance in data points. Variance is a measurement of how data points differ from the mean.

> A Toy Example

Before we attempt to apply PCA to the sklearn breast cancer dataset, let's use a toy example to gain some intuition about PCA. The code below generates a dataset with two features.

In [4]:
# let's generate a toy 2D dataset with a strong linear relationship
np.random.seed(42)
X_new = np.dot(np.random.rand(2, 2), np.random.randn(2, 200)).T
X_new = X_new - X_new.mean(axis=0)  # center data

Let's plot the data to visualize the two features.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.5)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Generated Toy Data')
plt.axis('equal')
plt.grid(False)
plt.show()

Next, let's explore how to reduce our 2D data to 1D. Imagine we pick a line and "flatten" all the data onto it. This process involves projecting each data point onto the line. We aim to understand this by visualizing how data points are moved to the line and measuring the "error" of this projection, which is the distance each point has to travel to reach the line. Ideally we want a pick a line that explains the most variance, so the distance these all these points have to move is the shortest.

In [ ]:
# define a line for projection: y = mx + c
# choose m and c to fit your data's general direction...
# (you can play around with slope and intercept values this but remember to return to original values before continuing on:
# m = 1
# c = 0
m = 1  # slope
c = 0  # intercept

# calculate projection points on the line for each X_new point (i.e. the distance each point must travel)
X_line = np.linspace(X_new[:, 0].min(), X_new[:, 0].max(), 100)
Y_line = m * X_line + c
X_proj = (m * (X_new[:, 1] - c) + X_new[:, 0]) / (m**2 + 1)
Y_proj = (m**2 * X_new[:, 1] + m * X_new[:, 0] + c) / (m**2 + 1)

# plot data (same as above!)
plt.figure(figsize=(8, 6))
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.5, label='Original Data')
# plot the line (line we defined with slope and intercept)
plt.plot(X_line, Y_line, color='red', label='Projection Line')
# plot projections (our distances!)
for i in range(len(X_new)):
    plt.plot([X_new[i, 0], X_proj[i]], [X_new[i, 1], Y_proj[i]], 'k--')
plt.scatter(X_proj, Y_proj, color='green', label='Projected Points', alpha=0.5)

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Projection onto a Line')
plt.legend()
plt.axis('equal')
plt.grid(False)
plt.show()

Now, we'll project our original data onto a different line, one that is orthogonal to the first line we used. By doing this, we aim to show that not all projections are equally useful. Specifically, we want to compare the effectiveness of these two lines in capturing the variance of the original data. The idea is to illustrate why one projection might be better than another by looking at how much distance points have to move during the projection, which directly relates to how well the line captures the data's variance.

To make this comparison, we'll calculate the average distance points must travel to be projected onto this new line. Then, we can compare this average distance with the one from the first projection to see which line results in a lower average distance, indicating a more efficient capture of the data's variance.

In [ ]:
m = -1 / m  # slope of the new line, orthogonal to the first line
c = Y_proj.mean() - m * X_proj.mean()  # adjusting intercept to pass through the mean of projected points

# define new projection line
X_line_new = np.linspace(X_new[:, 0].min(), X_new[:, 0].max(), 100)
Y_line_new = m * X_line_new + c

# calculate new projection points
X_proj_new = (m * (X_new[:, 1] - c) + X_new[:, 0]) / (m**2 + 1)
Y_proj_new = (m**2 * X_new[:, 1] + m * X_new[:, 0] + c) / (m**2 + 1)

# calculate distances for the new projection
distances_new = np.sqrt((X_new[:, 0] - X_proj_new)**2 + (X_new[:, 1] - Y_proj_new)**2)

# new projection
plt.figure(figsize=(8, 6))
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.5, label='Original Data')
plt.plot(X_line_new, Y_line_new, color='red', label='Orthogonal Projection Line')
for i in range(len(X_new)):
    plt.plot([X_new[i, 0], X_proj_new[i]], [X_new[i, 1], Y_proj_new[i]], 'k--')
plt.scatter(X_proj_new, Y_proj_new, color='green', label='Projected Points', alpha=0.5)

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Orthogonal Projection onto a Line')
plt.legend()
plt.axis('equal')
plt.grid(False)
plt.show()

In [ ]:
#let's compare the average distances
average_distance_original = np.mean(np.sqrt((X_new[:, 0] - X_proj)**2 + (X_new[:, 1] - Y_proj)**2))
average_distance_new = np.mean(distances_new)

average_distance_original, average_distance_new

---
##### **Q2: Which direction was better to project the data points?**

<span style="background-color: #FFD700">**Write Answer Here**</span>

---

After exploring how projections onto different lines can vary in their effectiveness at capturing data variance, we're now ready to introduce Principal Component Analysis (PCA) into our discussion.

Essentially, PCA is a systematic method to identify the **most informative projection directions**—those that maximize variance and, hence, capture the data's inherent structure. Unlike our earlier manual explorations with lines, PCA automates this search and quantifies the importance of each direction.

In this section, we apply PCA to our toy data to visually understand how PCA identifies principal components. These components are the best lines we can project our data onto if we want to preserve as much information as possible.

The principal components show us not just any lines but the ones along which the variance of our data is maximized. This is basically a way to automatically find the optimal projection line, with a data-driven approach.

Let's take a look at how PCA works on our toy data. We'll plot the original data points and overlay the principal components as vectors. These vectors represent the directions of maximum variance, with their length indicative of the variance magnitude explained by each component.

In [ ]:
# StandardScaler is used to standardize features by removing the means and adjust to the variance
from sklearn.preprocessing import StandardScaler

# use StandardScaler to scale the features
# We will explain why scaling data is necessary in PCA later
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

# apply PCA to toy data
pca_2d = PCA(n_components=2)
pca_2d.fit(X_scaled)

# plot the original toy data
plt.figure(figsize=(8, 6))
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.5)

#add the principal components to the plot as direction and magnitude arrows
for length, vector in zip(pca_2d.explained_variance_, pca_2d.components_):
    v = vector * 3 * np.sqrt(length)
    plt.quiver(pca_2d.mean_[0], pca_2d.mean_[1], v[0], v[1], angles='xy', scale_units='xy', scale=1, color='red')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Principal Components on Toy Data')
plt.axis('equal')
plt.grid(True)
plt.show()

### Explaining Scaling in PCA

In our implementation of PCA earlier, prior to applying PCA, we scaled the data, now let's explain why.

Scaling is a critical preprocessing step for PCA, especially when your features have different units and scales. This is because PCA looks for directions of maximum variance to identify principal components. Variance is heavily influenced by the scale of the features; If one feature has a much larger scale than others, PCA might misleadingly consider that feature more important, not because of any intrinsic property of the data, but simply due to its larger numeric ranges.

Let's look at a simple example. Given a dataset with two features:
-  Feature 1: ranges from 0 to 1
-  Feature 2: ranges from 0 to 100
    
Without scaling, PCA can overemphasize the variance along the direction of larger numerical ranges, which is Feature 2. If plotting the PCA, the principal component would lie almost entirely along the axis of Feature 2.
Consequently, the variance captured by PCA is overwhelming influenced by Feature 2 and overshadowing any contribution from Feature 1. This leads to bias towards Feature 2.
Feature 1 may be equally important, however, PCA ignores Feature 1 because of its range is small and influence on variance is small before scaling.

With scaling, PCA gives a more balanced and accurate representation of the data's intrinsic structure, highlighting the true directions that maximize variance. Scaling before applying PCA ensures that all features contribute equally to the analysis.

### Standardize data

Now let's go back to the breast cancer dataset that is introduced at the beginning.

Let's prepare the data for PCA.

In [10]:
X = data.data
y = data.target
feature_names = data.feature_names

In [ ]:
print(feature_names)

Starting with standardizing the data:

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Apply PCA

In [12]:
#we're setting the number of components to 2 to get a 2D visualization by changing value of n_components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
#look at the PCA Results
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='plasma', edgecolor='k', s=40)
plt.title('2D PCA of Breast Cancer Dataset')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(label='Target')
plt.show()

As we coloured the samples by target ID, we can see that the data appears separable based on the target label. We could draw a line on this graph to separate cancerous and non-cancerous samples (our malignant vs. benign labelled histopathologies).

Right now we just did PCA with two components, and set the first two components as the axis for the visualization.

How much variation did these two components explain?

In [ ]:
explained_variance = pca.explained_variance_ratio_
print(f"Explained Variance Ratio: {explained_variance}")

Here we can see the explained variance ratio for the first two components, the only two components in our PCA.

What if we did PCA with more than 2 components?

In [ ]:
#keep more components and see how much variance they explain
pca = PCA(n_components=10)  # Keeping 10 components
X_pca = pca.fit_transform(X_scaled)

# variance Explained
explained_variance = pca.explained_variance_ratio_
print(f"Explained Variance Ratio: {explained_variance}")

---
##### **Q3. Do you notice anything about how much variance is explained by the extra components? Is there a trend?**

<span style="background-color: #FFD700">**Write Answer Here**</span>

---

Now let's take a look at the total cumulative variance explained by these components, when we have 10 components in the PCA.

(Note that there are many ways to visualize explained variance of principal components in PCA and this is just one way)

In [ ]:
#cumulative Variance Explained
cumulative_variance = np.cumsum(explained_variance)
plt.figure(figsize=(8, 5))
plt.bar(range(1, 11), explained_variance, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(1, 11), cumulative_variance, where='mid', label='Cumulative explained variance')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

**Answer the question below.**

##### **Q4. How does adding components change the cumulative explained variance?**

<span style="background-color: #FFD700">**Write Answer Here**</span>

---

*How do we pick how many principal components to explain a dataset?*

Sometimes we pick the number of principal components to keep based on our goal of visualization: if we want to view a reduced dimensionality version of the dataset, we can only use maximum three dimensions. However, sometimes we want to use PCA for dimensionality reduction without a final goal of visualization. So how many components do we keep?

We can take a look at:
[Nature paper on PCA](https://www.nature.com/articles/s43586-022-00184-w)

This paper gives us the "elbow rule", which suggests looking for a point where the explained variance by additional dimensions starts decreasing linearly. If we look at the cumulative explained variance plot earlier, we can draw a curve to fit the decline of each components contribution to explained variance. The point in that curve that is the 'elbow' of the curve, suggests where to cut off the number of components needed to explain the dimensionality of the data. If the first two components explain significantly more variance than the subsequent ones, this indicates that the dataset is essentially two-dimensional (almost all the variance can be explained by the first two principal components). Thus, a two-dimensional PCA effectively captures the major variance in data, with remaining dimensions likely representing noise or less significant variation.




Here we draw a curve to find an "elbow" over our cumulative explained variance plot.

In [ ]:
# draw a curve that fits the tops of the histograms
plt.figure(figsize=(8, 5))
plt.bar(range(1, 11), explained_variance, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(1, 11), cumulative_variance, where='mid', label='Cumulative explained variance')

# plot curve over the individual explained variance
plt.plot(range(1, 11), explained_variance, color='red', marker='o', linestyle='-',
         label='Curve of Individual Variance')

plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

How much variance should be explained by the first principal components to be "valid"? i.e. how do we know PCA is actually explaining a good amount of variance?

*Answer*:

If we are unable to explain a "significant" amount of variance in a "reasonable" number of components, maybe PCA is not the best tool to reduce the dimensionality of the data. In fact, if a significant amount of variance cannot be captured by the initial components, it may suggest that the data contains complex, non-linear relationships that PCA, a linear method, cannot adequately represent.

What is "significant" variance and what is a "reasonable" number of components?
The definition of "significant" variance and a "reasonable" number of components can vary depending on the specific context and domain of the data.

*   General Rule of Thumb: a significant amount of variance is often considered to be around 70% or more of the total variance in the data
*    One approach to determining a reasonable number of components is the elbow method, which involves plotting the explained variance against the number of components and looking for the "elbow" in the curve of the graph
* Another approach to determining a "reasonable" number of components is based on visualization: reducing a dataset to three or fewer dimensions can enable visualization in 2D or 3D space, which might be desirable for certain analyses despite lower total explained variance.




Let's take a look at how adding components changes the cumulative explained variance.

In [ ]:
# function to perform PCA and return explained variance
def perform_pca(n_components, data):
    pca = PCA(n_components=n_components)
    pca.fit(data)
    return pca.explained_variance_ratio_, pca.components_

# exploring the effect of different numbers of components
components_to_try = [2, 5, 10, 15, 20, 25, 30]
explained_variances = []
for n in components_to_try:
    explained_variance, _ = perform_pca(n, X_scaled)
    explained_variances.append(sum(explained_variance))

# plotting the total explained variance against the number of components
plt.figure(figsize=(10, 6))
plt.plot(components_to_try, explained_variances, marker='o')
plt.title('Effect of Number of Components on Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Total Explained Variance')
plt.grid(True)
plt.show()

### Component Analysis

*How do we ensure our dimensionality reduction keeps the most important information?*

There are many techniques for dimensionality reduction that aim to preserve the most important information. In the case of PCA, we will be learning how to preserve the features that "explain the most variance". This means, we will be summarizing the data using whatever features explain the most differences in our dataset, which ideally represents the labels of the classification task.

Usually in dimensionality reduction, when we transform higher-dimensional data (data with a lot of features) into a lower dimensional space (the summary), each dimension in the lower-dimensional space represents a combination of the original features.

When we transform higher-dimensional data (data with a lot of features) into a lower dimensional space (the summary), each dimension in the lower-dimensional space represents a combination of the original features.

Let's took a look at what composition of features these components have.

In [ ]:
n_components = 10
_, components = perform_pca(n_components, X_scaled)

# create a DataFrame for better visualization
components_df = pd.DataFrame(components, columns=feature_names)

# visualize the composition of the first few components
plt.figure(figsize=(14, 8))
for i in range(5):
    plt.subplot(3, 2, i+1)
    components_df.loc[i].plot(kind='bar')
    plt.title(f'Component {i+1}')
    plt.ylabel('Weight')
    plt.tight_layout()
plt.show()

Intuitively, a positive weight in a component means that the feature contributes to that principal component.

A negative weight in a PCA component means that the feature inversely correlates with that principal component. Each principal component is a linear combination of the original features, and the sign (positive or negative) indicates the direction of the correlation between the component and the feature. Negative weights suggest that as the feature value increases, the data points move in the opposite direction along that principal component axis.

In [ ]:
# look at the composition of the first two principal components.
pca_components = pd.DataFrame(pca.components_, columns=feature_names)
print(pca_components.head(2))

The PCA component weights reveal the relative importance of each feature in the dataset's variance. For instance, if texture has a high weight in the first principal component, it suggests that texture variation significantly contributes to distinguishing between different observations in the dataset. This might reflect the biological reality where texture variations in histopathology images are crucial for distinguishing between benign and malignant breast cancer types.



## Conclusion
PCA is a powerful tool for dimensionality reduction in high-dimensional datasets. Each component of PCA is a weighted combination of the features in the dataset and represent the most significant patterns or gradients of variation in the data. Often, these components may also capture important variations that well seperate points by some target value, making PCA a tool for effective visualization and generating useful insights into the data's underlying structure.

As a reminder, some good practices for using PCA include:
1. Scale your data before running PCA!
2. Use the explained variance ratio to determine how many components to retain (usually we want above 70%).

Despite the benefits, one must also be careful regarding the interpretation of PCA. While the components discovered by PCA are informative and useful in many cases, it is also true that many real-world datasets contain *non-linear* patterns that PCA is unable to capture. For these, we need more advanced algorithms that we will cover in Week 3.



## Graded Exercises (5 Marks)

##### **GQ1. (1 marks) Diabetes Data PCA**
> In the cell block below, we have loaded in a new dataset, the Diabetes Dataset, and printed out the description. Run PCA on this dataset and plot out the first two components of the PCA. Color the points by `y`.

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Load the diabetes dataset
data = load_diabetes()
X = data.data
y = data.target

In [ ]:
### Your Code Here

---
##### **GQ2. (2 marks) For the PCA you ran above, what is the explained variance of the first two components? Based on this, do you think two components is sufficient to capture the true structures of the data? Justify your answer.**


In [ ]:
## YOUR CODE HERE

Write Your Answer Here

---
##### **GQ3. (2 marks) Visualize the composition for the first two components. Which features are important for these components? Looking at the PCA components, do the important features make sense based on the value of y above? Justify your answer.**


In [ ]:
## YOUR CODE HERE

## Optional

### Data Reconstruction from PCA

In this section, we delve into the concept of data reconstruction from PCA-transformed data, aiming to solidify our understanding of the dimensionality reduction process and its implications.

In [ ]:
# reconstruct data from the reduced dataset (using inverse transformation)
X_reconstructed = pca.inverse_transform(X_pca)

Dimensionality reduction through PCA simplifies the dataset by projecting it onto a lower-dimensional space defined by the principal components. This process, while beneficial for reducing complexity and computational costs, entails a loss of information. By reconstructing the original data from its PCA-transformed version, we can visually and quantitatively assess what "information loss" actually means.

### Compare Original Data and Reconstructed Data

Reconstruction involves reversing the PCA transformation, which provides us with a version of our dataset that retains only the variance captured by the selected principal components. Comparing this reconstructed dataset to our original dataset allows us to directly observe the consequences of dimensionality reduction.

In [ ]:
# compare the reconstructed dataset to the original dataset
# This can give us an idea of how much information we've managed to preserve or lose.
error = np.mean((X_scaled - X_reconstructed) ** 2, axis=1)
print(f"Mean reconstruction error: {np.mean(error)}")

### Reconstruct with less components

Let's take a look at what happens when we do PCA with less components. We can see how much information is lost in the reconstruction, or how much worse our reconstruction error is:

In [ ]:
pca = PCA(n_components=2)  # Keeping 2 components
X_pca = pca.fit_transform(X_scaled)
X_reconstructed = pca.inverse_transform(X_pca)
error = np.mean((X_scaled - X_reconstructed) ** 2, axis=1)
print(f"Mean reconstruction error: {np.mean(error)}")

 How does the reconstruction error affect our confidence in the PCA-transformed data?

*Answer*:

The reconstruction error directly impacts our confidence in the PCA-transformed data. A lower reconstruction error indicates that the PCA-transformed data retains a significant portion of the original dataset's variance, meaning we can trust the reduced dataset to accurately represent the original data's structure and patterns. Conversely, a high reconstruction error suggests that much of the original data's information is lost in the transformation, potentially leading to inaccurate analyses or conclusions based on the PCA-transformed data.

Bonus: Can you think of any datasets for which PCA might not be the ideal method for dimensionality reduction?

*Answer*:



* Non-Linear Data Structures: Remember when we couldn't explain a "significant" amount of variance in a "reasonable" number of components using PCA? PCA is fundamentally a linear method: it assumes that the principal components can linearly capture the dataset's variance. If the data contains complex, non-linear relationships, PCA may fail to identify these patterns, and techniques designed to capture non-linearity (like t-SNE or UMAP) might be more appropriate.
* Interpretable Component Requirements: If the application requires easily interpretable components, PCA might not be ideal. PCA components are linear combinations of all original features, which can be challenging to interpret, especially in domains where specific features have distinct meanings.

